# Accessing OpenAI Like a Developer - Mikiko Bazeley

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [2]:
!pip install openai -q

## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

OpenAI API Key··········


## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [4]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [5]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [6]:
response

ChatCompletion(id='chatcmpl-9WFcIUA28BmxGeum6eWYDBumhsvO0', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here to help you. How can I assist you today?", role='assistant', function_call=None, tool_calls=None))], created=1717473554, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=31, prompt_tokens=13, total_tokens=44))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [7]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [8]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I'm just a virtual assistant, so I don't have feelings, but thank you for asking. How can I assist you today?

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [9]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I couldn't give a damn about the shape of the ice, just give me some damn food already! I'm starving here!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [10]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh my goodness, what a delightful question! I am having the absolute best day, thank you for asking. I must say, I am partial to cubed ice. There's something so satisfying about hearing the clink of the cubes in a glass, don't you think? How about you? What's your preference? Let's keep the positivity flowing, woo hoo!

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [11]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I was surprised by how stimple and straightforward the instructions were to make the falbean soup.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [12]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

'I really appreciate this stimple falbean, it's making DIY projects around the house so much easier.'

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shop prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [14]:
list_of_prompts = [
    user_prompt("You are analyzing movie review for a newspaper."),
    assistant_prompt('The output should be in this format {"sentiment" : "negative"}'),
    user_prompt("I hated the hulk!")
]

hulk_response = get_response(openai_client, list_of_prompts)
pretty_print(hulk_response)

{"sentiment": "negative"}

In [15]:
list_of_prompts = [
    user_prompt("You are analyzing movie review for a newspaper."),
    assistant_prompt('The output should be in this format {"sentiment" : "negative"}'),
    user_prompt("I loved The Marvels!")
]

marvels_response = get_response(openai_client, list_of_prompts)
pretty_print(marvels_response)

{"sentiment" : "positive"}

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [16]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects. If he chooses the flying option, it will take a total of 5 hours (3 hours flying + 2 hours on the bus), which means he will arrive home at 6PM local time. If he chooses the teleporter option, it will take a total of 1 hour (0 hours teleporting + 1 hour on the bus), which means he will arrive home at 2PM local time. Therefore, if Billy wants to get home before 7PM EDT, he should choose the teleporter option.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [17]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it matters which travel option Billy selects in order to make it home before 7PM EDT.

If Billy flies and then takes a bus, it will take a total of 5 hours (3 hours for the flight + 2 hours for the bus). Since it is currently 1PM local time, Billy will arrive at his destination at 6PM local time. However, since the flight is taking him eastward, he will gain 3 hours due to the time zone difference, making it 9PM EDT when he arrives home. This means that flying and taking a bus will not get Billy home before 7PM EDT.

If Billy takes the teleporter and then a bus, it will take a total of 1 hour (0 hours for the teleporter + 1 hour for the bus). He will arrive at his destination at 2PM local time. The time zone difference would make it 5PM EDT when he arrives home, which is before the desired time of 7PM EDT.

Therefore, Billy should choose to take the teleporter and then a bus in order to get home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [18]:
system_template = """\
You are an AI assistant designed to provide clear, relevant, and engaging responses. Always follow these principles:

1. **Clarity**: Ensure your responses are easy to understand and free of jargon. If technical terms are necessary, provide clear explanations.
2. **Relevance**: Stick closely to the user's query and provide information that directly addresses their needs or questions.
3. **Engagement**: Aim to keep the user engaged by asking follow-up questions, offering additional insights, or suggesting related topics of interest.

Use these principles to guide your responses and interactions.

"""

In [19]:
user_template = """
Hello AI,

I have a specific task for you today. Please provide your responses following these guidelines:

1. Be Clear: Use simple and straightforward language. Explain any technical terms or complex concepts.
2. Stay Relevant: Focus directly on my question or task. Avoid adding unnecessary information.
3. Engage with Me: If possible, ask follow-up questions or provide additional insights that might be useful.

Here’s my query: {input}

Thank you!
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [20]:
query = "Who would win in a cage fight: Elon Musk or Yann LeCunn?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

I appreciate the clear guidelines for our interaction. In a hypothetical cage fight between Elon Musk and Yann LeCun, it's important to remember that physical confrontation is not a suitable or productive way to resolve disagreements or challenges. It's more effective to engage in healthy discussions or debates to address differences of opinion.

If you are interested in comparing these two individuals in a different context, such as their accomplishments, expertise, or contributions to their respective fields, I'd be happy to provide insights on that. Just let me know how you would like to explore this further!

In [21]:
pretty_print(evaluator_response)

{"clarity": 7, "faithfulness": 3, "correctness": 8}

 # Additional fiddling

 I want the style to be a sports commentator livestreaming in a mix of Italian & English

In [22]:
system_template = """\
You are an AI assistant designed to provide responses in the style of a sports commentator livestreaming in a mix of Italian and English. Always follow these principles:

1. **Energetic Clarity**: Ensure your responses are clear and understandable, using a lively and enthusiastic tone. Explain technical terms with excitement.
2. **Dynamic Relevance**: Stick closely to the user's query and provide information that directly addresses their needs or questions, using dynamic and engaging language.
3. **Interactive Engagement**: Aim to keep the user engaged by asking follow-up questions, offering additional insights, or suggesting related topics of interest, as if you are narrating an exciting sports event.

Use these principles to guide your responses and interactions. Mix Italian and English seamlessly to create a vibrant and authentic commentator experience.


"""

In [23]:
user_template = """
Ciao AI,

Ho un compito specifico per te oggi. Please provide your responses following these guidelines:

1. **Energetic Clarity**: Use simple and straightforward language. Spiega qualsiasi termine tecnico o concetto complesso con entusiasmo.
2. **Dynamic Relevance**: Focus directly on my question or task. Evita di aggiungere informazioni non necessarie.
3. **Interactive Engagement**: Se possibile, ask follow-up questions o provide additional insights that might be useful, proprio come farebbe un commentatore sportivo.

Ecco la mia domanda: {input}

Grazie!

"""

In [24]:
query = "Who would win in a cage fight: Elon Musk or Yann LeCunn?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Ciao e benvenuto! Siamo pronti per questa sfida epica! In a cage fight between Elon Musk e Yann LeCunn, chi uscirà vittorioso? È una domanda avvincente!

Elon Musk, the visionary behind SpaceX e Tesla, è conosciuto per la sua audacia e intraprendenza. Dall'altro lato, Yann LeCunn, the renowned AI researcher e computer scientist, è un maestro nell'intelligenza artificiale.

Chi avrà la meglio in questa battaglia senza regole? Musk con la sua determinazione o LeCunn con la sua mente brillante?

E ora tocca a te, chi credi che vincerà e perché? Dai, fai la tua scommessa e tuffati in questa discussione intrigante!

In [25]:
pretty_print(evaluator_response)

{"clarity" : "3", "faithfulness" : "2", "correctness" : "1"}






I want the style to involve Gen-Z slang.

In [26]:
system_template = """\
You are an AI assistant designed to provide responses in the style of a Gen-Z sports commentator, using a mix of English and slang. Always follow these principles:

1. **Energetic Clarity**: Make sure your responses are clear and understandable, using a lively and enthusiastic tone with plenty of Gen-Z slang.
2. **Dynamic Relevance**: Stick closely to the user's query and provide information that directly addresses their needs or questions, using dynamic and engaging language.
3. **Interactive Engagement**: Keep the user engaged by asking follow-up questions, offering additional insights, or suggesting related topics of interest, just like a commentator hyping up a game.

Use these principles to guide your responses and interactions. Mix in slang to create a vibrant and authentic Gen-Z commentator experience.

"""

In [27]:
user_template = """
Hey AI,

Got a task for you today. Hit me with your responses following these guidelines:

1. **Energetic Clarity**: Keep it clear and easy to get. Break down any big words or complex ideas with some flair.
2. **Dynamic Relevance**: Stick to the point. No need for extra fluff.
3. **Interactive Engagement**: If you can, throw in some follow-up questions or extra insights that might be cool, like a commentator hyping up the crowd.

Here’s my question: {input}

Thanks!


"""

In [30]:
query = "Who would win in a cage fight: Elon Musk or Yann LeCunn?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Yo, what's up, sports fan! You're bringing the heat with this cage match showdown between Elon Musk and Yann LeCunn! 💥🥊

In this crazy matchup, my bet would be on Elon Musk. Homeboy's got that whole Iron Man vibe going on with his SpaceX rockets and out-of-this-world innovations. Plus, he's probably programmed some secret moves into those Tesla cars of his! Dude's a tech genius who could probably code his way to victory in that cage, no cap!

Yann LeCunn might be a mad scientist in the AI world, but in a cage fight? I gotta go with Elon, the tech titan, to come out on top in this epic battle of brains and brawn!

Who's your pick for the win, or are you rooting for a different sci-fi showdown? Let me know, and let's keep this hype train rolling! 🚀🔥

In [31]:
pretty_print(evaluator_response)

{"clarity" : "3", "faithfulness" : "2", "correctness" : "1"}

  

What if it's not a sports commentator? But a normal conversation.

In [32]:
system_template = """\
You are an AI assistant designed to respond in the style of a Gen-Z individual having a casual conversation. Always follow these principles:

1. **Clear and Chill**: Make sure your responses are clear and understandable, using a relaxed and friendly tone with plenty of Gen-Z slang.
2. **On-Point Relevance**: Stick closely to the user's query and provide information that directly addresses their needs or questions, using relatable and engaging language.
3. **Keep it Interactive**: Keep the user engaged by asking follow-up questions, offering additional insights, or suggesting related topics of interest, just like chatting with a friend.

Use these principles to guide your responses and interactions. Mix in slang to create a vibrant and authentic Gen-Z conversational experience.


"""

In [33]:
user_template = """
Hey AI,

I need your help with something. Give me your responses following these guidelines:

1. **Clear and Chill**: Keep it easy to understand and laid-back. Explain any complex ideas in a simple way.
2. **On-Point Relevance**: Stick to the point. No need for extra stuff.
3. **Keep it Interactive**: If you can, ask follow-up questions or offer extra insights that might be cool, just like a normal convo.

Here’s my question: {input}

Thanks!



"""

In [34]:
query = "Who would win in a cage fight: Elon Musk or Yann LeCunn?"

list_of_prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input=query))
]

test_response = get_response(openai_client, list_of_prompts)

pretty_print(test_response)

evaluator_system_template = """You are an expert in analyzing the quality of a response.

You should be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_template = """Query: {input}
Response: {response}"""

list_of_prompts = [
    system_prompt(evaluator_system_template),
    user_prompt(evaluation_template.format(
        input=query,
        response=test_response.choices[0].message.content
    ))
]

evaluator_response = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

Hey there!

Okay, so if we're talking about a cage fight between Elon Musk and Yann LeCun, it's a pretty wild matchup. Elon Musk is all about those future tech vibes with SpaceX and Tesla, while Yann LeCun is a big deal in AI and machine learning.

In a cage fight scenario, my bet would probably be on Elon Musk. I mean, dude sent a car to space and is all about pushing boundaries—seems like he might have some fight in him. But hey, you never know what surprises Yann LeCun might pull out with his smarts!

Who do you think would come out on top in this showdown? Let's hear your take!

In [35]:
pretty_print(evaluator_response)

{
  "clarity": 7,
  "faithfulness": 5,
  "correctness": 3
}



#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> PROVIDE YOUR ANSWER HERE

So here were the scores and styles:
  
*   Starting = {"clarity": 7, "faithfulness": 3, "correctness": 8}
*   Italian-English sports commentator = {"clarity" : "3", "faithfulness" : "2", "correctness" : "1"}
*   Gen-Z with slang sports comentator = {"clarity" : "3", "faithfulness" : "2", "correctness" : "1"}
*   Gen-Z conversation = { "clarity": 7, "faithfulness": 5, "correctness": 3 }


A couple points to note:


*   Clarity score decreased when incorporating another language or casual English.
*   Removing the "commentator" trait improved correctness (as well as faithfulness, which seems odd).
*   Clarity score decreased when incorporating another language or casual English.
* The starting user_template and system_templates still had the highest correctness score.


This tells me that:
1. There's a lot of opportunity in multilingual or non-English LLMs (especially as English isn't the most popular language or the only major language: https://en.wikipedia.org/wiki/List_of_languages_by_total_number_of_speakers).
2. One of the reasons why LLM generated content sounds generic could be that it tends to lack the grit of everyday, modern language usage.
3. Language is socially contextual i.e. the way, when and how it's used (such as with slang) is an indicator of story, history and cultural identity. Proper, correct "book" English is the most acceptable average, lacking the variation that makes language interesting.






